In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Exploring reward funcion options

## **Creating a test framework**

In [89]:
class TestGenerator:
    def __init__(
                self,
                number_of_users_to_generate:int,
                total_views_average:int,
                total_unique_views_average:int,
                total_prompts_created_average:int,
                total_curator_views_average:int,
                average_curator_score:int = 10 ,
                number_of_curator_views:int = 5,
                reward_function: object = None,
                curator_views_compact:bool = True,
                curator_scores_compact:bool = True,
                total_views_compact:bool = True,
                unique_views_compact:bool = True,
                prompts_compact:bool = True,
                compact_factor:float = 0.2,
                extreme_factor:int = 20,
                extreme_perc:float = 0.1,
                curator_scores_extreme:bool = False,
                total_views_extreme:bool = False,
                unique_views_extreme:bool = False,
                prompts_extreme:bool = False,

                ):
        '''
        number_of_users_to_generate: self explanatory
        total_views_average: (average) total number of views from all blocks created by the user
        total_unique_views_average: (average) number of unique viewers from all the blacks generated by the user
        total_prompts_created_average: (average) number of blocks referencing all the user generated blocks (could be sandbox additions too)
        number_of_curator_views_average: (average) number of curators that referenced the user's blocks to their curation (could be sandbox additions too)
        reward_function: function that takes some the parameters above and calculates a final score
        X__compact: defines if the numbers will be cluster around the averages (above) or will be totally spread
        compact_factor: the percentage where the random values will be generate from: average_Value * +- compact_factor
        extreme_factor: factor by what I multiply when values are not compacted to create the range from 
                        [1, averege * max I want for the metric to go]
        extreme_perc: percentage of users that should get an extreme value
        X__extreme: determines for that variable if a specigic percentage of users should get a very extreme value

        '''
        self.compact_factor = compact_factor
        self.number_of_users_to_genereate = number_of_users_to_generate
        self.total_views_average = total_views_average
        self.total_unique_views_average = total_unique_views_average
        self.total_prompts_created_average = total_prompts_created_average
        self.total_views_compact = total_views_compact
        self.unique_views_compact = unique_views_compact
        self.prompts_compact = prompts_compact
        self.total_curator_views_average = total_curator_views_average
        self.number_of_curator_views = number_of_curator_views
        self.average_curator_score = average_curator_score
        self.reward_function = reward_function
        self.extreme_factor = extreme_factor
        self.curator_views_compact = curator_views_compact
        self.curator_scores_compact = curator_scores_compact
        self.simulated_users = None
        self.extreme_perc = extreme_perc
        self.curator_scores_extreme = curator_scores_extreme
        self.total_views_extreme = total_views_extreme
        self.unique_views_extreme = unique_views_extreme
        self.prompts_extreme = prompts_extreme


    def generate_users(self):
        simulated_users = []
        tviews_ext_counter = 0
        uvies_ext_counter = 0
        prompts_ext_counter = 0
        curator_ext_counter = 0

        for x in range(self.number_of_users_to_genereate):
            user_data = {'id': x}

            #initializing flag variables
            user_data['is_outlier'] = False
            user_data['is_total_views_outlier'] = False
            user_data['is_unique_views_outlier'] = False
            user_data['is_prompts_outlier'] = False
            user_data['is_curator_views_outlier'] = False

            #generating total views
            if self.total_views_extreme and tviews_ext_counter < (self.number_of_users_to_genereate * self.extreme_perc):
                user_data['total_views'] = self.generate_total_views() * self.extreme_factor
                tviews_ext_counter += 1
                user_data['is_outlier'] = True
                user_data['is_total_views_outlier'] = True
            else:
                user_data['total_views'] = self.generate_total_views()
           
           
            #generating unique views
            if self.unique_views_extreme and uvies_ext_counter < (self.number_of_users_to_genereate * self.extreme_perc):
                user_data['unique_views'] = self.generate_unique_views() * self.extreme_factor
                uvies_ext_counter += 1
                user_data['is_outlier'] = True
                user_data['is_unique_views_outlier'] = True
            else:
                user_data['unique_views'] = self.generate_unique_views()
     

            #generating number of prompts
            if self.prompts_extreme and prompts_ext_counter < (self.number_of_users_to_genereate * self.extreme_perc):
                user_data['prompts_created'] = self.generate_prompts_creation() * self.extreme_factor
                prompts_ext_counter += 1
                user_data['is_outlier'] = True
                user_data['is_prompts_outlier'] = True
            else:
                user_data['prompts_created'] = self.generate_prompts_creation()
     

            #generating curator view weighted average
            if self.curator_scores_extreme and curator_ext_counter < (self.number_of_users_to_genereate * self.extreme_perc):
                user_data['curator_views_weighted'] = self.generate_curator_views() * self.extreme_factor
                curator_ext_counter += 1
                user_data['is_outlier'] = True
                user_data['is_curator_views_outlier'] = True
            else:
                user_data['curator_views_weighted'] = self.generate_curator_views()
         

            simulated_users.append(user_data)
        
        self.simulated_users = simulated_users
        
        return simulated_users
    
    def generate_user_with_scores(self, reward_function_weights:list = [0.1, 0.2, 0.3, 0.4]):
        if self.simulated_users == None:
            self.generate_users()
        
        final_data = []
        for user in self.simulated_users:
            score_function_parameters = {
                'parameters_weights' : reward_function_weights,
                **user
            }
            del score_function_parameters["id"]
            del score_function_parameters["is_outlier"]
            del score_function_parameters["is_curator_views_outlier"]
            del score_function_parameters["is_prompts_outlier"]
            del score_function_parameters["is_unique_views_outlier"]
            del score_function_parameters["is_total_views_outlier"]
            

            user_score = self.reward_function(**score_function_parameters)
            user['final_score'] = user_score

            final_data.append(user)

        return final_data
    
    def generate_total_views(self):
        if self.total_views_compact:
            delta = self.total_views_average * self.compact_factor
            return random.randrange(self.total_views_average - delta, self.total_views_average + delta)
        else:
            return  random.randrange( 1 , self.total_views_average * self.extreme_factor)
    
    def generate_unique_views(self):
        if self.unique_views_compact:
            delta = self.total_unique_views_average * self.compact_factor
            return random.randrange(self.total_unique_views_average - delta, self.total_unique_views_average + delta)
        else:
            return  random.randrange( 1 , self.total_unique_views_average * self.extreme_factor)
    
    def generate_prompts_creation(self):
        if self.prompts_compact:
            delta = self.total_prompts_created_average * self.compact_factor
            return random.randrange(self.total_prompts_created_average - delta, self.total_prompts_created_average + delta)
        else:
            return  random.randrange( 1 , self.total_prompts_created_average * self.extreme_factor)
    
    def generate_curator_views(self):
        products = 0
        scores = 0
        for _ in range(self.number_of_curator_views):
            # assigning the curator a score
            if self.curator_scores_compact:
                delta = self.average_curator_score * self.compact_factor
                curator_score = random.randrange(self.average_curator_score - delta, self.average_curator_score + delta)
            else: 
                curator_score = random.randrange( 1 , self.average_curator_score * self.extreme_factor)

            # assigning the number of views for that curator
            if self.curator_views_compact:
                delta = self.total_curator_views_average * self.compact_factor
                curator_view = random.randrange(self.total_curator_views_average - delta, self.total_curator_views_average + delta)
            else: 
                curator_view= random.randrange( 1 , self.total_curator_views_average * self.extreme_factor)
            
            products += curator_score * curator_view
            scores += curator_score
        
        return products / scores # returns weighted average of views by curator score
        

## **Reward Functions**

Rewards functions have always the same parameters:

- **List of weights for each of the variables**
- **total_views** : total number of views for all blocks created by the user
- **unique_views**: total number of unique views generated for all blocks created by the user
- **prompts_cretead**: number of blocks created by the user that were reference from another user (or added to other user sandbox)
- **curator_views_weighted**: weighted average views by score of each curator that viewed the user blocks

### **Linear weighted sum**

In [41]:
def linear_sum(parameters_weights:list,
                total_views:int, 
                unique_views:int,
                prompts_created:int,
                curator_views_weighted:dict,
                ):
    '''
    total_views: total numbers of queries that saw/used that content, 
    unique_views: number of unique users that saw/used the content through the queries,
    prompts_created: number of other user's content created on top of the user content,
    curator_views_weighted: {user_score: total_views}
    * parameters_weights: list of weights for each pameters, range should be [0, 100] and parameter_weights.sum() == 100
                            [ w_total_views, w_unique_view, w_prompts_created, w_views_by_score] in this order
    
    returns: user final score for the epoch
    '''
    # assigning weights
    w_total_views = parameters_weights[0]
    w_unique_view = parameters_weights[1]
    w_prompts_created = parameters_weights[2]
    w_views_by_score = parameters_weights[3]


    final_result = (total_views * w_total_views) \
                    + (unique_views * w_unique_view) \
                    + (prompts_created * w_prompts_created ) + \
                    + (w_views_by_score * curator_views_weighted)

    return final_result

### **Log weighted sum**

In [92]:
def log_sum(parameters_weights:list,
                total_views:int, 
                unique_views:int,
                prompts_created:int,
                curator_views_weighted:dict,
                ):
    '''
    total_views: total numbers of queries that saw/used that content, 
    unique_views: number of unique users that saw/used the content through the queries,
    prompts_created: number of other user's content created on top of the user content,
    curator_views_weighted: {user_score: total_views}
    * parameters_weights: list of weights for each pameters, range should be [0, 100] and parameter_weights.sum() == 100
                            [ w_total_views, w_unique_view, w_prompts_created, w_views_by_score] in this order
    
    returns: user final score for the epoch
    '''
    # assigning weights
    w_total_views = parameters_weights[0]
    w_unique_view = parameters_weights[1]
    w_prompts_created = parameters_weights[2]
    w_views_by_score = parameters_weights[3]


    final_result =  np.log(total_views * w_total_views) \
                    + np.log(unique_views * w_unique_view) \
                    + (prompts_created * w_prompts_created ) + \
                    + (w_views_by_score * curator_views_weighted)

    return final_result

## **Simulating**

#### Similar users

**Linear weighted**

In [90]:
test_parameters = {
    'number_of_users_to_generate': 10,
    'total_views_average':100,
    'total_unique_views_average':30,
    'total_prompts_created_average':10,
    'total_curator_views_average': 15,
    'average_curator_score': 10 ,
    'number_of_curator_views':  5,
    'reward_function': linear_sum,
    'curator_views_compact':True,
    'curator_scores_compact':True,
    'total_views_compact':True,
    'unique_views_compact':True,
    'prompts_compact':True,
    'compact_factor':0.2,
    'extreme_factor':20,
    'extreme_perc': 0.1,
    'curator_scores_extreme':False,
    'total_views_extreme': True,
    'unique_views_extreme': False,
    'prompts_extreme': False,
}

sim_test = TestGenerator(**test_parameters)

sim_dt = pd.DataFrame(sim_test.generate_user_with_scores())

sim_dt.sort_values('final_score', ascending=False)

,id,is_outlier,is_total_views_outlier,is_unique_views_outlier,is_prompts_outlier,is_curator_views_outlier,total_views,unique_views,prompts_created,curator_views_weighted,final_score
0,0,True,True,False,False,False,2020,32,8,13.977273,216.390909
9,9,False,False,False,False,False,101,32,10,16.209302,25.983721
5,5,False,False,False,False,False,103,31,11,12.574468,24.829787
7,7,False,False,False,False,False,98,32,9,14.770833,24.808333
6,6,False,False,False,False,False,91,31,11,14.862745,24.545098
1,1,False,False,False,False,False,105,25,11,13.795918,24.318367
8,8,False,False,False,False,False,95,30,8,15.000000,23.900000
2,2,False,False,False,False,False,105,24,10,13.187500,23.575000
3,3,False,False,False,False,False,97,26,11,13.403846,23.561538
4,4,False,False,False,False,False,99,25,9,14.695652,23.478261


**Log weighted**

In [103]:
test_parameters = {
    'number_of_users_to_generate': 15,
    'total_views_average':100,
    'total_unique_views_average':30,
    'total_prompts_created_average':10,
    'total_curator_views_average': 15,
    'average_curator_score': 10 ,
    'number_of_curator_views':  5,
    'reward_function': log_sum,
    'curator_views_compact':True,
    'curator_scores_compact':True,
    'total_views_compact':True,
    'unique_views_compact':True,
    'prompts_compact':False,
    'compact_factor':0.2,
    'extreme_factor':20,
    'extreme_perc': 0.1,
    'curator_scores_extreme':False,
    'total_views_extreme': True,
    'unique_views_extreme': True,
    'prompts_extreme': False,
}

sim_test = TestGenerator(**test_parameters)

sim_dt = pd.DataFrame(sim_test.generate_user_with_scores())

sim_dt.sort_values('final_score', ascending=False)

,id,is_outlier,is_total_views_outlier,is_unique_views_outlier,is_prompts_outlier,is_curator_views_outlier,total_views,unique_views,prompts_created,curator_views_weighted,final_score
14,14,False,False,False,False,False,86,29,191,12.777778,66.320731
13,13,False,False,False,False,False,110,29,183,15.729167,65.347420
2,2,False,False,False,False,False,115,27,171,14.222222,61.117635
10,10,False,False,False,False,False,86,35,147,15.020000,54.205672
1,1,True,True,True,False,False,1740,580,126,14.795918,53.631013
4,4,False,False,False,False,False,93,27,106,14.808511,41.639818
5,5,False,False,False,False,False,97,28,86,15.822222,36.123781
7,7,False,False,False,False,False,117,34,53,15.590909,26.512875
3,3,False,False,False,False,False,92,24,46,14.489362,23.383564
9,9,False,False,False,False,False,81,28,44,14.660000,22.878631


#### Testing extremes 

#### Similar user scores

#### Similar user scores

#### Similar user scores